----------------------------------------
### 벡터검색 시나리오
이 시나리오는DB에 저장되어 있는 문서(pdf,hwpx)를 문장 임베딩 모델을 사용하여 유사 문서를 검색하는 데모입니다.

### 0. 파이썬 라이브러리 로딩 및 23ai 

In [1]:
# 23ai 접속 및 라이브러리 로딩
import oracledb
import oml
import pandas as pd
import import_ipynb
from IPython.display import display
from user_functions import get_conn_cursor, get_db_version, exec_sql, dml_sql, ddl_sql, clean_text

#print("Complete load libries")

# DB 접속
conn, cursor = get_conn_cursor("vector","vector","localhost:1521/freepdb1")

# DB 버전 확인
get_db_version(cursor)

,PRODUCT,VERSION_FULL
0,Oracle Database 23ai Free,23.8.0.25.04


### 1. 샘플 테이블(doc_store) 데이터 조회
- docno varchar(10) primary key
- doc_dept varchar(40)
- doc_cr_date varchar(20)
- doc_name varchar2(200)
- doc blob

In [2]:
# 샘플(doc_store) 테이블 데이터 조회

q1 = """ select docno, doc_dept, doc_cr_date, doc_name from doc_store order by 2, 1 """
exec_sql(cursor,q1)

,DOCNO,DOC_DEPT,DOC_CR_DATE,DOC_NAME
0,80-1,국방부,20250531,국방정책실장 미국 상하원 대표단 면담
1,80-2,국방부,20250704,화생방사 국제 화학방호교육
2,80-3,국방부,20257011,미국 B-52H 전략폭격기 전개하 한미일 공중훈 시행
3,80-4,국방부,20250715,2025일본방위백서기술내용관련항의
4,80-5,국방부,20250721,사이버 작전사령부 美 사이버플래그훈련 참가
5,10-1,기획재정부,20250710,광역 교통망 확충 3개 사업 예타 통과로 대도교통혼잡 해소기대
6,10-2,기획재정부,20250716,임신공무원하루2시간모성보호시간사용보장
7,10-3,기획재정부,20250717,하반기관세대응수출금융설명회개최
8,10-4,기획재정부,20250718,공급망등 경제안보 분야에서 통상 환경변화를 고려한 대응전략 고도화
9,20-1,행정안전부,20250715,지방공무원 관리자 역량 끌어올릴 역량교육 전문가 양성한다


### 2. 임베딩 모델 및 쿼리 임베딩 확인
- 이 데모에서는 DB에 저장된 엠베딩 모델을 사용합니다.
- 쿼리 임베딩 방법은 오라클 벡터 함수(dbms_vector_chain)을 사용하여 DB에 저장된 문서에서 텍스트 추출(utl_to_text)->청킹(utl_to_chunks)->임베딩(utl_to_embedding)을 단일 라인 코드로 실행

In [3]:
# DB관리중인 ONNX 모델 조회
managed_models = 'SELECT MODEL_NAME, MINING_FUNCTION, ALGORITHM, ALGORITHM_TYPE, (MODEL_SIZE/1024/1024) MB FROM user_mining_models ORDER BY MODEL_NAME'
print("DB관리중인 ONNX 모델")
exec_sql(cursor,managed_models)

DB관리중인 ONNX 모델


,MODEL_NAME,MINING_FUNCTION,ALGORITHM,ALGORITHM_TYPE,MB
0,MULTILINGUAL_E5_SMALL,EMBEDDING,ONNX,NATIVE,117.378383
1,PARAPHRASE_MULTILINGUAL_MPNET_BASE_V2,EMBEDDING,ONNX,NATIVE,270.086040
2,VIT_BASE_PATCH16_224,EMBEDDING,ONNX,NATIVE,327.618084
3,VIT_BASE_PATCH16_224_IMG,EMBEDDING,ONNX,NATIVE,292.181850
4,VIT_BASE_PATCH16_224_TXT,EMBEDDING,ONNX,NATIVE,119.933461


In [12]:
# 임베딩 벡터 데이터 확인
print("임베딩 모델 : PARAPHRASE_MULTILINGUAL_MPNET_BASE_V2")
get_vector = """select dt.docno as docno,et.embed_id as chunk_id, et.embed_data as chunked_data, to_vector(et.embed_vector) embed_vector
from doc_store dt,
   dbms_vector_chain.utl_to_embeddings(
       dbms_vector_chain.utl_to_chunks(dbms_vector_chain.utl_to_text(dt.doc), 
       json('{ "by" : "words","max" : "100", "overlap" : "10","split" : "newline", "language" : "korean","normalize": "all"}')),
       json('{"provider":"database", "model":"PARAPHRASE_MULTILINGUAL_MPNET_BASE_V2"}')) t,
   JSON_TABLE(t.column_value, '$' columns (embed_id number,embed_data varchar2(4000), embed_vector CLOB)) et
where dt.docno = :get_docno
fetch first 10 rows only
"""
pd.set_option('display.max_colwidth', 100)
result = exec_sql(cursor,get_vector,{"get_docno":"10-3"})
result = clean_text(result)
result['CHUNKED_DATA']=result['CHUNKED_DATA'].str.replace('\n', '', regex=False)
display(result)

임베딩 모델 : PARAPHRASE_MULTILINGUAL_MPNET_BASE_V2


,DOCNO,CHUNK_ID,CHUNKED_DATA,EMBED_VECTOR
0,10-3,1,"보도시점 2025. 7. 17.(목) 16:00 배포 2025. 7. 17.(목) 14:00 하반기 관세대응 수출금융 설명회 개최 - 원스톱 수출･수주지원단, 6개 정책금융...","[-0.008440015837550163, -0.014268845319747925, -0.007599708624184132, 0.006944766268134117, 0.01..."
1,10-3,2,"(Fin-eX: 수출입은행, 무역 보험공사, 중소벤처기업진흥공단, 신용보증기금, 기술보증기금, IBK기업은행)와 140여개 기업이 참석했다. 이번 설명회는 통상환경 변화에 ...","[-0.04923214390873909, -0.032898783683776855, -0.003978660795837641, -0.010259585455060005, 0.03..."
2,10-3,3,"(수은), 통상리스크대응긴급자금(중진공)등 미국 관세 대응에 필요한 자금지원 방안도 종합적으로 안내되었다. 지원단은 앞으로도 우리 수출기업들이 美 통상정책의 불확실성에 신속...","[-0.0025694682262837887, -0.005203770473599434, -0.004503738135099411, -0.009448950178921223, 0...."
3,10-3,4,"불확실한통상환경속에서중소‧중견기업대상수출금융· 보증등맞춤형수출금융정보제공및활용독려추진 ㅇ 6개정책금융기관*이공동으로참여하는수출금융설명회로,기업 수요에맞는금융지원정보를효율적‧...","[-0.0004947329289279878, 0.021538984030485153, -0.010122720152139664, -0.01979854889214039, 0.02..."
4,10-3,5,"(10분)인사말씀 및 행사순서 안내 무역협회14:10~16:10 (120분)기관별 주요 지원사업 소개 (기관당 20분: 발표 10분, Q&A 10분) * ➊수출입은행 → ➋...","[-0.0365537591278553, -0.010705417022109032, -0.004355018958449364, -0.005673778709024191, 0.022..."


### 3. 데이터 임베딩 및 인덱싱
- doc_store의 모든 문서의 텍스트를 추출, 워드 100개 단위로 청킹 후, 청크별로 벡터라이징(임베딩)하여 doc_store_chunks 테이블의 벡터 컬럼에 저장

In [6]:
# 기존 데이터 삭제
remove_old_data = "truncate table doc_store_chunks"
dml_sql(conn,cursor,remove_old_data)

'complete commit for DML'

In [7]:
# 임베딩 & 벡터 저장
load_vector = """insert into doc_store_chunks
select dt.docno as docno,et.embed_id as chunk_id, et.embed_data as chunked_data, to_vector(et.embed_vector) embed_vector
from doc_store dt,
   dbms_vector_chain.utl_to_embeddings(
       dbms_vector_chain.utl_to_chunks(dbms_vector_chain.utl_to_text(dt.doc), 
       json('{ "by" : "words","max" : "100", "overlap" : "10","split" : "newline", "language" : "korean","normalize": "all"}')),
       json('{"provider":"database", "model":"PARAPHRASE_MULTILINGUAL_MPNET_BASE_V2"}')) t,
   JSON_TABLE(t.column_value, '$' columns (embed_id number,embed_data varchar2(4000), embed_vector CLOB)) et
"""
print("벡터라이징 및 저장중입니다........")
dml_sql(conn,cursor,load_vector)

벡터라이징 및 저장중입니다........


'complete commit for DML'

In [8]:
# 벡터 인덱싱
ddl_sql(cursor,"drop index if exists doc_store_chunks_idx")

# 벡터 인덱싱
crIdx = """
create vector index doc_store_chunks_idx
on doc_store_chunks(embed_vector)
ORGANIZATION INMEMORY NEIGHBOR GRAPH
WITH DISTANCE COSINE
"""
ddl_sql(cursor,crIdx)

# 벡터 인덱싱 조회
idx_q = """select table_name, index_name, index_type, index_subtype, status 
           from user_indexes where 
           table_name = 'DOC_STORE_CHUNKS'"""
exec_sql(cursor,idx_q)

,TABLE_NAME,INDEX_NAME,INDEX_TYPE,INDEX_SUBTYPE,STATUS
0,DOC_STORE_CHUNKS,SYS_IL0000087960C00004$$,LOB,None,VALID
1,DOC_STORE_CHUNKS,PK_CHUNK,NORMAL,None,VALID
2,DOC_STORE_CHUNKS,DOC_STORE_CHUNKS_IDX,VECTOR,INMEMORY_NEIGHBOR_GRAPH_HNSW,VALID


### 4. 문서 유사성 검색

In [13]:
# 검색 문장 입력 및 검색
pd.set_option('display.max_colwidth', 200)
print("예: 고향사랑 기부 상반기 모금 결과")
user_sentence = input("찾으실 문장을 입력하세요:")

sim_q = """
Select T.docno,
D.doc_name, 
T.chunked_id,
T.chunked_data AS 문서내용
from doc_store_chunks T, doc_store D
where D.docno = T.docno
order by vector_distance(T.embed_vector, 
                         dbms_vector.utl_to_embedding(:u_sentence, 
                                                       json('{"provider":"database", "model":"PARAPHRASE_MULTILINGUAL_MPNET_BASE_V2"}')),
                         cosine)
fetch first 1 rows only
"""
sim_result = exec_sql(cursor,sim_q,{"u_sentence":user_sentence})
sim_result = clean_text(sim_result)
sim_result['문서내용']=sim_result['문서내용'].str.replace('\n', '', regex=False)
display(
    sim_result.head(10).style
    .set_properties(**{'text-align': 'left'})
        .set_table_styles([{'selector': 'th', 'props': [('text-align', 'left')]}])
)

예: 고향사랑 기부 상반기 모금 결과


찾으실 문장을 입력하세요: 고향사랑 기부 상반기 모금 결과


,DOCNO,DOC_NAME,CHUNKED_ID,문서내용
0,20-2,고향사랑기부로 불어 넣은 지역 활력 2025년상반기모금결과공개,1,"- 1 -보도자료보도시점(온라인) 2025. 7. 16.(수) 12:00 (지 면) 2025. 7. 17.(목) 조간고향사랑기부로 불어넣은 지역활력, 2025년 상반기 모금결과 공개 - 상반기 총 모금액 349억 원(전년대비 1.7배), 총 모금건수 28만 건(전년대비 1.9배) - 지정기부 시행(2024년 6월) 이후 누적 모금액 100억 원 돌파, 성공적으로 안착"
